In [ ]:
from tabulate import tabulate # Import the tabulate library for creating formatted tables.

class LibraryManagementSystem:
    def __init__(self):
        #file names for storing data and load existing data.
        self.books_file = 'books.txt'
        self.borrowers_file = 'borrowers.txt'
        self.checkouts_file = 'checkouts.txt'
        self.books = self.load_data(self.books_file) # Load books data from file.
        self.borrowers = self.load_data(self.borrowers_file) # Load borrowers data from file.
        self.checkouts = self.load_data(self.checkouts_file) # Load checkouts data from file.

    def load_data(self, filename):
        """
        Loads data from text files (books.txt, borrowers.txt, checkouts.txt)
        into the respective lists (books, borrowers, checkouts).
        """
        try:
            with open(filename, 'r') as file:
                 # Read lines from the file, strip whitespace, and split by comma
                data = [line.strip().split(',') for line in file.readlines()]
            return data
        except FileNotFoundError:
            return []  # Return an empty list if the file is not found.

    def save_data(self, filename, data):
        """
        Saves data from the respective lists (books, borrowers, checkouts)
        into text files (books.txt, borrowers.txt, checkouts.txt).
        """
        with open(filename, 'w') as file:
            for record in data:
                # Write each record to the file, joining elements with commas
                file.write(','.join(record) + '\n')

    def display_books(self):
        """
        Displays a list of all books in the library.
        """
        headers = ['ISBN','Title','Author',"Publication Year",'Availability']
        print(tabulate(self.books, headers=headers, tablefmt="grid"))

    def display_borrowers(self):
        """
        Displays a list of all borrowers in the library.
        """ 
        headers = ["Borrower ID", "Name","Contact Details"]
        print(tabulate(self.borrowers, headers=headers,tablefmt="grid"))
            
    def search_borrower_by_id(self, borrower_id):
        """
        Searches for a borrower by ID and returns the Borrower object if found.
        """
        for borrower in self.borrowers:
             #To verify that the entered number matches a number in the file
            if borrower[0] == borrower_id:
                return borrower
        return None

    def search_book_by_isbn(self, isbn):
        """
        Searches for a book by ISBN and returns the Book object if found.
        """
        for book in self.books:
            #To verify that the entered number matches a number in the file
            if book[0] == isbn:
                return book
        return None

    def add_new_borrower(self, borrower_id, name):
        """
        Adds a new borrower to the library system
        """
        if not self.search_borrower_by_id(borrower_id):
            # Append new borrower record and save the data
            self.borrowers.append([borrower_id, name])
            self.save_data(self.borrowers_file, self.borrowers)
            print(f'Borrower {name} with ID {borrower_id} added successfully.')
        else:
            print(f'Borrower with ID {borrower_id} already exists.')

    def update_book_information(self, isbn):
        """
        Search for the book by ISBN
        """
        
        book = self.search_book_by_isbn(isbn)
        if book:
            print('\t'.join(book))
            # Prompt user for field and new value
            field = input("Enter the field to update (title/author/publication year): ").lower()
            new_value = input("Enter the new value: ")
            # Determine the index of the field in the book record
            field_index = {'title': 1, 'author': 2, 'publication year': 3}.get(field, None)
            if field_index is not None:
                # Update the field value and save the data
                book[field_index] = new_value
                self.save_data(self.books_file, self.books)
                print('Book information updated successfully.')
            else:
                print('Invalid field.')
        else:
            print(f'Book with ISBN {isbn} not found.')

    def delete_book(self, isbn):
        """
        Delete a book from the library management system
        """
        book = self.search_book_by_isbn(isbn)
        if book:
            print('\t'.join(book))
            confirmation = input("Are you sure you want to delete this book? (yes/no): ").lower()
            # Confirm deletion with the user
            if confirmation == 'yes':
                # Remove the book record and update data files
                self.books.remove(book)
                self.save_data(self.books_file, self.books)
                # Remove corresponding checkouts
                self.checkouts = [checkout for checkout in self.checkouts if checkout[0] != isbn]
                self.save_data(self.checkouts_file, self.checkouts)
                print('Book deleted successfully.')
            else:
                print('Deletion canceled.')
        else:
            print(f'Book with ISBN {isbn} not found.')

    def add_new_book(self, isbn, title, author, publication_year):
        """
        Adding a new book to the library
        """
        if not self.search_book_by_isbn(isbn):
            # Append new book record and save the data
            self.books.append([isbn, title, author, publication_year, 'Available'])
            self.save_data(self.books_file, self.books)
            print(f'Book {title} with ISBN {isbn} added successfully.')
        else:
            print(f'Book with ISBN {isbn} already exists.')

    def checkout_book(self, isbn, borrower_id):
        """
        Borrowing a book from the library
        """
        book = self.search_book_by_isbn(isbn)
        borrower = self.search_borrower_by_id(borrower_id)
        if book and borrower:
            if book[4] == 'Available':
                # Add checkout record and update book availability
                self.checkouts.append([isbn, borrower_id])
                self.save_data(self.checkouts_file, self.checkouts)
                #To change the book's status from available to checked out
                book[4] = 'checked out'
                self.save_data(self.books_file, self.books)
                print(f'Book {book[1]} checked out successfully by {borrower[1]}.')
            else:
                #The message that will be output if the book is already checked out
                print(f'Book {book[1]} is not available for checkout.')
        else:
            print('Invalid ISBN or borrower ID.')

    def return_book(self, isbn, borrower_id):
        """
        Return a book to the library by ISBN and borrower ID.
    
        Parameters:
        - isbn (str): The ISBN of the book to be returned.
        - borrower_id (str): The ID of the borrower returning the book.
        """
        book = self.search_book_by_isbn(isbn)
        #To search for the borrower who wants to return the book
        borrower = self.search_borrower_by_id(borrower_id)
        if book and borrower:
            if [isbn, borrower_id] in self.checkouts:
                 # Remove checkout record and update book availability
                self.checkouts.remove([isbn, borrower_id])
                self.save_data(self.checkouts_file, self.checkouts)
                #To change the book's status from checked out to available 
                book[4] = 'Available'
                self.save_data(self.books_file, self.books)
                print(f'Book {book[1]} returned successfully by {borrower[1]}.')
            else:
                print(f'Book {book[1]} is not checked out by {borrower[1]}.')
        else:
            print('Invalid ISBN or borrower ID.')

    def list_books_checked_out_by_borrower(self, borrower_id):
        """
        List all books currently checked out by a specific borrower.
    
        Parameters:
        - borrower_id (str): The unique ID of the borrower whose checked-out books 
          are to be listed.
        """
        borrower = self.search_borrower_by_id(borrower_id)
        if borrower:
            # Filter checked out books for the specific borrower
            checked_out_books = [book for book in self.books if book[4] == 'checked out']
            borrower_checkouts = [book for book in checked_out_books if [book[0], borrower_id] in self.checkouts]
            if borrower_checkouts:
                # Display the list of list_books_checked_out_by_borrower using the tabulate library.
                headers = ["ISBN", "Title", "Author", "Publication Year", "Availability"]
                print(f'Books checked out by {borrower[1]}:')
                print(tabulate(borrower_checkouts, headers=headers, tablefmt="grid"))
            else:
                #The message that will be printed if no book is checked out by the borrower
                print(f'No books checked out by {borrower[1]}.')
        else:
            print('Invalid borrower ID.')

# Define the main function
def main():
    library = LibraryManagementSystem()

    while True:
         # Display the main menu 
        print("""
Library Management System
=========================
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit
=========================
""")
        #Take user input
        choice = input('Enter your choice: ')

        if choice == '1':
            library.display_books()
            #Call library_system.display books function if the user enters the number 1
        elif choice == '2':
            library.display_borrowers()
            #Call library_system.display_borrowers books function if the user enters the number 1
        elif choice == '3':
            borrower_id = input('Enter borrower ID: ')
            #The user is asked to enter an ID to be used to search for a borrower
            borrower = library.search_borrower_by_id(borrower_id)
            if borrower:
                #The output will be printed in table form
                headers = ["Borrower ID", "Name", "Contact Details"]
                print(tabulate([borrower], headers=headers, tablefmt="grid"))
            else:
                #The message that will be printed if the ID number is not exist in the system
                print(f'Borrower with ID {borrower_id} not found.')

        elif choice == '4':
            isbn = input('Enter book ISBN: ')
            #The user is asked to enter an ISBN to be used to search for a book
            book = library.search_book_by_isbn(isbn)
            if book:
                #The output will be printed in table form
                headers = ["ISBN", "Title", "Author", "Publication Year", "Availability"]
                print(tabulate([book], headers=headers, tablefmt="grid"))
            else:
                #The message that will be printed if the ISBN number is not exist in the system
                print(f'Book with ISBN {isbn} not found.')
        elif choice == '5':
            #The user was asked to enter the ID and browser to be added to the system
            borrower_id = input('Enter borrower ID (seven digits): ')
            name = input('Enter borrower name: ')
            library.add_new_borrower(borrower_id, name)
        elif choice == '6':
            #The user is asked to enter the ISBN. to updated the system information .
            isbn = input('Enter book ISBN: ')
            library.update_book_information(isbn)
        elif choice == '7':
            #The user is asked to enter the ISBN. To delete the book.
            isbn = input('Enter book ISBN: ')
            library.delete_book(isbn)
        elif choice == '8':
            #The user is asked to enter the number, title, author, and year of publication in order to add a book to the system
            isbn = input('Enter book ISBN (five-digit unique integer): ')
            title = input('Enter book title: ')
            author = input('Enter book author: ')
            publication_year = input('Enter publication year: ')
            library.add_new_book(isbn, title, author, publication_year)
        elif choice == '9':
            #The user is asked to enter the ISBN number for the book to be checked out by the borrower
            isbn = input('Enter book ISBN: ')
            borrower_id = input('Enter borrower ID: ')
            library.checkout_book(isbn, borrower_id)
        elif choice == '10':
            #The user is asked to enter the ISBN number in order to return the book that was checked out by the borrower
            isbn = input('Enter book ISBN: ')
            borrower_id = input('Enter borrower ID: ')
            library.return_book(isbn, borrower_id)
        elif choice == '11':
            #The user is asked to enter the borrower’s ID number so that the books that have been checked out will be printed
            borrower_id = input('Enter borrower ID: ')
            library.list_books_checked_out_by_borrower(borrower_id)
        elif choice == '12':
            print('Exiting the Library Management System. Goodbye!')
            ## Exit the program if the user chooses option 12
            break  
        else:
            #If the user enters a number except 1 to 12,
            print('Invalid choice. Please enter a number from 1 to 12.')


# Call the main function to start the program.
main()



Library Management System
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit



Enter your choice:  9
Enter book ISBN:  12345
Enter borrower ID:  1234567


Invalid ISBN or borrower ID.

Library Management System
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit



Enter your choice:  1


+--------+---------+----------+--------------------+----------------+
|   ISBN | Title   | Author   | Publication Year   | Availability   |
+========+=========+==========+====================+================+
|  12345 | John    | sfse     | 2000               | checked out    |
+--------+---------+----------+--------------------+----------------+
|  23456 | Doe     | 2021     | Not Available      |                |
+--------+---------+----------+--------------------+----------------+

Library Management System
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit



Enter your choice:  1


+--------+---------+----------+--------------------+----------------+
|   ISBN | Title   | Author   | Publication Year   | Availability   |
+========+=========+==========+====================+================+
|  12345 | John    | sfse     | 2000               | checked out    |
+--------+---------+----------+--------------------+----------------+
|  23456 | Doe     | 2021     | Not Available      |                |
+--------+---------+----------+--------------------+----------------+

Library Management System
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit



Enter your choice:  2


+---------------+--------+
|   Borrower ID | Name   |
+===============+========+
|       7654321 | peter  |
+---------------+--------+
|       1235687 | Amal   |
+---------------+--------+
|       1111111 | Alice  |
+---------------+--------+
|       0000001 | Bob    |
+---------------+--------+
|             6 | hxvx   |
+---------------+--------+

Library Management System
1. View list of books
2. View list of borrowers
3. Search for a borrower by ID
4. Search for a book by ISBN
5. Add a new borrower
6. Update book information
7. Delete a book
8. Add a new book
9. Checkout a book
10. Return a book
11. List books checked out by a borrower
12. Exit



## 